In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import accelerate
from transformers import pipeline

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )

Loading checkpoint shards: 100%|██████████| 8/8 [09:53<00:00, 74.14s/it]


In [3]:
file_path = "/home/ltnga/LawVN-Instructction-Gen/src/data/data_gen.json"
with open(file_path, 'r', encoding='utf-8') as f:
        data =json.load(f)

In [4]:
data['content'][0]

'Điều 34. Xử phạt, trừ điểm giấy phép lái xe của người điều khiển quá khổ giới hạn, xe quá tải trọng, xe bánh xích lưu hành đường bộ (kể cả xe ô tô chở hành khách)\n1. Phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với hành vi không thực hiện đúng quy định trong giấy phép lưu hành, trừ các hành vi vi phạm quy định tại điểm a, điểm b khoản 3; điểm b, điểm c khoản 4 Điều này.\n2. Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi điều khiển xe mà tổng trọng lượng (khối lượng toàn bộ) của xe vượt quá tải trọng cho phép của đường bộ trên 10% đến 20%, trừ trường hợp có giấy phép lưu hành còn giá trị sử dụng.\n3. Phạt tiền từ 8.000.000 đồng đến 10.000.000 đồng đối với một trong các hành vi vi phạm sau đây:\na) Chở hàng vượt khổ giới hạn của xe hoặc của đường bộ ghi trong giấy phép lưu hành;\nb) Điều khiển xe bánh xích tham gia giao thông không có giấy phép lưu hành hoặc có giấy phép lưu hành nhưng không còn giá trị sử dụng theo quy định hoặc lưu thông trực tiếp trên đường mà 

In [38]:
import re
from rapidfuzz import fuzz
def is_similar(q1: str, q2: str, threshold=70) -> bool:
    """
    Sử dụng fuzzy matching để so sánh hai câu hỏi.
    Nếu điểm tương đồng >= threshold, coi như chúng trùng ý.
    """
    ratio = fuzz.ratio(q1.lower(), q2.lower())
    return ratio >= threshold


def extract_qa_pairs(generated_text: str):
    """
    Tách tất cả cặp hỏi-đáp từ một đoạn văn bản đầu ra của mô hình.
    Trả về danh sách dict có keys: 'question' và 'answer'.
    """
    pattern = r"Câu hỏi:\s*(.*?)\s*Trả lời:\s*(.*?)(?=\s*Câu hỏi:|$)"
    matches = re.findall(pattern, generated_text, flags=re.DOTALL)

    results = []
    for q_content, a_content in matches:
        q_clean = clean_text(q_content)
        a_clean = clean_text(a_content)
        results.append({
            "question": q_clean,
            "answer": a_clean
        })
    return results


def clean_text(text: str) -> str:
    """
    Loại bỏ các ký tự không mong muốn như '---', '###', dấu xuống dòng thừa, 
    khoảng trắng thừa ở đầu/cuối, v.v...
    """
    text = re.sub(r"---+", "", text)   
    text = re.sub(r"#+", "", text)     
    text = re.sub(r"\s*\n\s*", " ", text) 
    text = text.strip()  
    return text

In [8]:
prompt_template = """
Bạn là một chuyên gia phân tích văn bản pháp luật Việt Nam. Nhiệm vụ của bạn là tạo các cặp câu hỏi và trả lời chuyên sâu, rõ ràng, phù hợp với nội dung và ngữ cảnh. Để đảm bảo chất lượng, vui lòng tuân thủ các quy tắc sau:

---

### Văn bản cung cấp:
{context}

---

### Quy tắc tạo **Câu hỏi**:
1. **Loại câu hỏi cần tạo**:
   - Định nghĩa pháp lý hoặc thuật ngữ quan trọng trong văn bản.
   - Phạm vi áp dụng hoặc đối tượng được điều chỉnh.
   - Quyền, nghĩa vụ, hoặc trách nhiệm của các bên liên quan.
   - Quy trình, thủ tục hoặc điều kiện áp dụng.
   - Chế tài xử lý hoặc biện pháp khắc phục hậu quả.
   - Mỗi câu hỏi không quá 20 từ.

2. **Yêu cầu nội dung câu hỏi**:
   - Không sử dụng các tham chiếu cụ thể .
   - Tập trung vào các quy định hoặc điểm chính của văn bản.
   - Sử dụng ngôn ngữ dễ hiểu, không quá chuyên môn hóa.
   - Rõ ràng, súc tích, phản ánh đúng nội dung trọng tâm.
   - Tính ứng dụng cao và liên quan trực tiếp.
   

---

### Quy tắc tạo **Câu trả lời**:
1. **Nội dung trả lời**:
   - Cung cấp thông tin rõ ràng, đầy đủ, xúc tích.
   - Liên kết các quy định với các khía cạnh khác nếu liên quan.
   - Không cần trích tham chiếu cụ thể  từ văn bản gốc.
   - Mỗi câu hỏi không quá 50 từ.

2. **Hình thức trình bày**:
   - Trình bày hệ thống, dễ hiểu, mạch lạc.
   - Duy trì tính khách quan, tránh suy diễn.
   - Ngôn ngữ pháp lý chuyên nghiệp nhưng dễ tiếp cận.
   - Tính ứng dụng cao, phù hợp với người dùng phổ thông.

---

### Định dạng yêu cầu:
- **Câu hỏi:** Bắt đầu bằng "Câu hỏi:".
- **Câu trả lời:** Bắt đầu bằng "Trả lời:".
- Mỗi cặp câu hỏi-trả lời được phân tách bằng một dòng trống.
- Giữ phong cách chuyên nghiệp nhưng thân thiện với người dùng phổ thông.

---

### Ví dụ mẫu:
Câu hỏi: Các hình thức xử phạt vi phạm hành chính trong giao thông đường bộ là gì?

Trả lời: Đối với mỗi hành vi vi phạm hành chính về trật tự, an toàn giao thông trong lĩnh vực giao thông đường bộ, cá nhân, tổ chức vi phạm phải chịu một trong các hình thức xử phạt chính sau đây:
a) Cảnh cáo;
b) Phạt tiền;
c) Tịch thu phương tiện được sử dụng để vi phạm hành chính.
---

Bây giờ, hãy tạo **một cặp câu hỏi và câu trả lời duy nhất** dựa trên văn bản đã cung cấp. Đảm bảo rằng câu trả lời không chứa câu hỏi tiếp theo.


    """

In [9]:
import json

qa_pairs = []
existing_questions = [] 
num_qa_pairs = 40

for _ in range(num_qa_pairs):
    
    prompt = prompt_template.format(context=str(data['content']))
    
   
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1000,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_content = response.split(prompt)[-1].strip()
    
    
    pairs = extract_qa_pairs(generated_content)

    if not pairs:
        # Nếu không có cặp Q&A nào
        qa_pairs.append({
                "question": "",
                "answer": "",
                
                })
    else:
        # Kiểm tra trùng lặp bằng fuzzy matching
        for pair in pairs:
            q_current = pair['question']
            # Kiểm tra xem q_current có "giống" câu hỏi nào trong existing_questions
            is_duplicate = any(is_similar(q_current, q_exist) for q_exist in existing_questions)
                    
            # Nếu không bị xem là giống, ta thêm vào danh sách
            if not is_duplicate:
                existing_questions.append(q_current)
                qa_pairs.append(pair)

output_file = "result.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

print(f"Đã lưu {len(qa_pairs)} cặp Q&A vào file {output_file}")

KeyError: '\n    "qas"'